## Description:
这个笔记本主要是先把YoutubeDNN召回跑起来， 采用的数据集是data_process下面的train_data.csv， 这个是用户历史行为数据+用户画像数据， 模型的话先调用包实现， 这里面主要完成的步骤是：
1. 导入数据， 划分出测试集来， 最后一天用户id进行评估
2. 用训练集的数据构造YoutubeDNN的数据集出来， 这里采用滑窗的方式构造， 但由于有些序列很长，尝试通过滑动步长优化
3. 构造YoutubeDNN的输入， 由于是调用deepmatch的YoutubeDNN，所以需要把输入特殊构造
4. 建立YoutubeDNN模型训练
5. 从YoutubeDNN中拿到用户embedding和doc的embedding，保存起来
6. 用第11天的测试集，进行评估，看看YoutubeDNN的召回效果

In [1]:
import os
import time
import pickle
import random
from datetime import datetime
import collections

import numpy as np
import pandas as pd

from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.preprocessing import LabelEncoder

# 从utils里面导入函数
from utils import gen_data_set, gen_model_input, train_youtube_model
from utils import get_embeddings, get_youtube_recall_res

import warnings
warnings.filterwarnings('ignore')
#os.environ["TF_CPP_MIN_LOG_LEVEL"] = "3"  # 1显示所有信息， 2显示error和warnings, 3显示error

DeepCTR version 0.9.0 detected. Your version is 0.8.2.
Use `pip install -U deepctr` to upgrade.Changelog: https://github.com/shenweichen/DeepCTR/releases/tag/v0.9.0


## 1. 导入数据，划分数据集

In [2]:
data_path = 'data_process'
data = pd.read_csv(os.path.join(data_path, 'train_data.csv'), index_col=0, parse_dates=['expo_time'])

In [3]:
data.shape

(3939989, 18)

In [5]:
latest_time = data['expo_time'].max()
# 添加example_age字段
data['example_age'] = (pd.to_datetime(latest_time) - data['expo_time'])
# 转换成小时的形式， 上面两式子相减是pandas的timedelta类型， 只有days和seconds属性
data['example_age'] = data['example_age'].apply(lambda x: x.days*24 + x.seconds//3600)

# 归一化
minmax = MinMaxScaler()
data['example_age'] = minmax.fit_transform(data['example_age'].values.reshape(-1, 1))

In [6]:
data.head()

,user_id,article_id,expo_time,net_status,flush_nums,exop_position,click,duration,device,os,province,city,age,gender,ctime,img_num,cat_1,cat_2,example_age
0,1000541010,464467760,2021-06-30 09:57:14,2,0,13,1,28,V2054A,Android,上海,上海,A_0_24,female,2021-06-29 14:46:43,3.0,娱乐,娱乐/港台明星,0.946108
1,1000541010,463850913,2021-06-30 09:57:14,2,0,15,0,0,V2054A,Android,上海,上海,A_0_24,female,2021-06-27 22:29:13,11.0,时尚,时尚/女性时尚,0.946108
2,1000541010,464022440,2021-06-30 09:57:14,2,0,17,0,0,V2054A,Android,上海,上海,A_0_24,female,2021-06-28 12:22:54,7.0,农村,农村/农业资讯,0.946108
3,1000541010,464586545,2021-06-30 09:58:31,2,1,20,0,0,V2054A,Android,上海,上海,A_0_24,female,2021-06-29 13:25:06,5.0,娱乐,娱乐/港台明星,0.946108
4,1000541010,465352885,2021-07-03 18:13:03,5,0,18,0,0,V2054A,Android,上海,上海,A_0_24,female,2021-07-02 10:43:51,18.0,娱乐,娱乐/港台明星,0.461078


In [7]:
# 选择出需要用到的列
use_cols = ['user_id', 'article_id', 'expo_time', 'net_status', 'exop_position', 'duration', 'device', 'city', 'age', 'gender', 'example_age', 'click']
data_new = data[use_cols]

In [8]:
data_new['exop_position'].unique().shape

(1158,)

## 划分测试集和训练集
* 训练集， 每个用户的历史点击，去掉最后一次
* 测试集， 每个用户的最后一次点击

In [9]:
# 按照用户分组，然后把最后一个item拿出来
click_df = data_new[data_new['click']==1]

In [10]:
def get_hist_and_last_click(all_click):
    all_click = all_click.sort_values(by=['user_id', 'expo_time'])
    click_last_df = all_click.groupby('user_id').tail(1)
    
    # 如果用户只有一个点击，hist为空了，会导致训练的时候这个用户不可见，此时默认泄露一下
    def hist_func(user_df):
        if len(user_df) == 1:
            return user_df
        else:
            return user_df[:-1]

    click_hist_df = all_click.groupby('user_id').apply(hist_func).reset_index(drop=True)

    return click_hist_df, click_last_df

In [11]:
user_click_hist_df, user_click_last_df = get_hist_and_last_click(click_df)

## 2. YoutubeDNN召回

In [12]:
def youtubednn_recall(data, topk=200, embedding_dim=8, his_seq_maxlen=50, negsample=0,
                      batch_size=64, epochs=1, verbose=1, validation_split=0.0):
    """通过YouTubeDNN模型，计算用户向量和文章向量
    param: data: 用户日志数据
    topk: 对于每个用户，召回多少篇文章
    """
    user_id_raw = data[['user_id']].drop_duplicates('user_id')
    doc_id_raw = data[['article_id']].drop_duplicates('article_id')
    
    # 类别数据编码   
    base_features = ['user_id', 'article_id', 'city', 'age', 'gender']
    feature_max_idx = {}
    for f in base_features:
        lbe = LabelEncoder()
        data[f] = lbe.fit_transform(data[f])
        feature_max_idx[f] = data[f].max() + 1
        
    # 构建用户id词典和doc的id词典，方便从用户idx找到原始的id
    user_id_enc = data[['user_id']].drop_duplicates('user_id')
    doc_id_enc = data[['article_id']].drop_duplicates('article_id')
    user_idx_2_rawid = dict(zip(user_id_enc['user_id'], user_id_raw['user_id']))
    doc_idx_2_rawid = dict(zip(doc_id_enc['article_id'], doc_id_raw['article_id']))
    
    # 保存下每篇文章的被点击数量， 方便后面高热文章的打压
    doc_clicked_count_df = data.groupby('article_id')['click'].apply(lambda x: x.count()).reset_index()
    doc_clicked_count_dict = dict(zip(doc_clicked_count_df['article_id'], doc_clicked_count_df['click']))

    train_set, test_set = gen_data_set(data, doc_clicked_count_dict, negsample, control_users=False)
    
    # 构造youtubeDNN模型的输入
    train_model_input, train_label = gen_model_input(train_set, his_seq_maxlen)
    test_model_input, test_label = gen_model_input(test_set, his_seq_maxlen)
    
    # 构建模型并完成训练
    model = train_youtube_model(train_model_input, train_label, embedding_dim, feature_max_idx, his_seq_maxlen, batch_size, epochs, verbose, validation_split)
    
    # 获得用户embedding和doc的embedding， 并进行保存
    user_embs, doc_embs = get_embeddings(model, test_model_input, user_idx_2_rawid, doc_idx_2_rawid)
    
    # 对每个用户，拿到召回结果并返回回来
    user_recall_doc_dict = get_youtube_recall_res(user_embs, doc_embs, user_idx_2_rawid, doc_idx_2_rawid, topk)
    
    return user_recall_doc_dict

In [13]:
user_recall_doc_dict = youtubednn_recall(user_click_hist_df, negsample=3)

100%|████████████████████████████████████████████████████████████████████████████| 20000/20000 [50:14<00:00,  6.64it/s]


Train on 1904804 samples
1904804/1904804 [==============================] - 3974s 2ms/sample - loss: 0.7390


In [14]:
len(user_recall_doc_dict)

20000

## 评估

In [15]:
user_click_last_df.shape

(20000, 12)

In [16]:
# 依次评估召回的前10, 20, 30, 40, 50个文章中的击中率
def metrics_recall(user_recall_items_dict, trn_last_click_df, topk=100):
    last_click_item_dict = dict(zip(trn_last_click_df['user_id'], trn_last_click_df['article_id']))
    user_num = len(user_recall_items_dict)
    
    for k in range(50, topk+1, 50):
        hit_num = 0
        for user, item_list in user_recall_items_dict.items():
            if user in last_click_item_dict:
                # 获取前k个召回的结果
                tmp_recall_items = [x[0] for x in user_recall_items_dict[user][:k]]
                if last_click_item_dict[user] in set(tmp_recall_items):
                    hit_num += 1
        
        hit_rate = round(hit_num * 1.0 / user_num, 5)
        print(' topk: ', k, ' : ', 'hit_num: ', hit_num, 'hit_rate: ', hit_rate, 'user_num : ', user_num)

In [17]:
metrics_recall(user_recall_doc_dict, user_click_last_df, topk=200)

 topk:  50  :  hit_num:  22 hit_rate:  0.0011 user_num :  20000
 topk:  100  :  hit_num:  37 hit_rate:  0.00185 user_num :  20000
 topk:  150  :  hit_num:  68 hit_rate:  0.0034 user_num :  20000
 topk:  200  :  hit_num:  99 hit_rate:  0.00495 user_num :  20000
